# Collection from aisixiang.com - political/philosophical scientific articles

ONLY PARTIALLY CLEANED AS OF 21 OCT 
BUT UPLOADED AS THIS WILL HAPPEN SOON (and uploading creates pressure to do so!)


In [1]:
import pandas as pd

import bs4
from bs4 import BeautifulSoup as soup

from urllib.request import urlopen as uReq
import urllib.request
from fake_useragent import UserAgent

import ssl
import time
import random

# Get collection URLS etc

In [2]:
ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
ua = UserAgent(verify_ssl=False) #cache=False # seems this option was removed
user_agent = ua.random
headers = {'User-Agent': user_agent}

In [4]:
base_url = 'https://www.aisixiang.com'
search_url = "/data/search.php?lanmu=" #n
pagination = '&page=' #n

In [5]:
# This may require some website recon, maybe make a dict, to make more automatic for end users
lanmu_list = ['677', '678', '676', '682', '683', '680', '681', '684', '679', '685', '688', '703', '482'] # econ, socio[1], peda[1]

In [6]:
def make_request(url):
    page = urllib.request.Request(url, headers=headers) 
    uClient = uReq(page)
    page_html = uClient.read()
    uClient.close()
    page_soup = soup(page_html, 'lxml')
    return page_soup

In [7]:
def get_details(page_soup):
    time.sleep(1)
    result_list = page_soup.find('div', class_= 'search_list')
    url_title = result_list.find_all('a') #('a', {'class': 'dihuias'})
    dates = result_list.find_all('span')
    #urls = page_soup.find_all('li') #('a', {'class': 'dihuias'})
    for u in url_title:
        item = u['href']
        item = base_url + item
        url_list.append(item)
    for t in url_title:
        item = t.text
        title_list.append(item)
    for d in dates:
        item_d = d.text
        date_list.append(item_d)

In [ ]:
url_list = []
title_list = []
date_list = []

counter = 1
#max_page = 2
for lanmu in lanmu_list:
    print('Searching: ' + lanmu)
    initial_page = base_url + search_url + lanmu + pagination + str(counter)
    page_soup = make_request(initial_page)

    #max page
    page_list = page_soup.find('div', class_= 'list_page')
    n_pages = page_list.find_all('a') 
    max_page = int(n_pages[-1]['href'].split('=')[-1])
    print('Pages: ' + str(max_page))

    while counter <= max_page:
        search_page = base_url + search_url + lanmu + pagination + str(counter)
        page_soup = make_request(search_page)
        print('Searching ' + search_page)
        
        get_details(page_soup)

        if counter == max_page:
            counter = 0
            print('Next catagory ... ')
            break
        
        time.sleep(1)
        counter +=1

url_list_filter = [u for u in url_list if 'lanmu' not in u]
title_list_filter = [u for u in title_list if '[理论法学]' not in u]

df = pd.DataFrame(list(zip(url_list_filter, date_list, title_list_filter)), columns =['url', 'date', 'title'])

In [ ]:
PATH = './DATA/URLS/'

df.to_csv(PATH + 'aisixiang_urls.csv', index = False)

# Load here

In [ ]:
# LOAD
df = pd.read_csv(PATH + 'aisixiang_urls.csv')

In [ ]:
df.date.min()

In [ ]:
df = df[df['date'] >= '2017-01-01']

In [ ]:
len(df)

# Get Full Texts

In [ ]:
text_list = []

In [ ]:
text_list

In [ ]:
def connect(index, url):

    print('Fetching ' + str(index))
    page_soup = make_request(url)        
    
    nap_time = random.uniform(.2, 1.5)
    print('Waiting: ' + str(nap_time))
    time.sleep(nap_time)

    text = page_soup.find('div', id='content')
    text_list.append(text.text) 
    #counter +=1

In [ ]:
counter = 1
for index, row in df.iterrows():
    try:
        connect(index, row.url)
    except:
        print('Connection error...sleeping for 2 minutes')
        time.sleep(120)
        print('Retrying...')
        connect(index, row.url)
print(len(text_list))

In [ ]:
df.tail(2)

In [ ]:
df['text'] = text_list #pd.DataFrame(url_list, text_list)

In [ ]:
df.reset_index(inplace=True)

In [ ]:
OUT_PATH = './DATA/'

df.to_csv(OUT_PATH + 'aisixiang_full.csv', index = False)